# Download and Inspect the Collection

The dataset was created from the Chronicling America collection — over 21 million digitized newspaper pages (1756–1963) curated by the Library of Congress and NEH. They used 39,330 pages (1800–1920), representing 53 US states, to ensure wide geographic and temporal coverage.

Source: https://dl.acm.org/doi/pdf/10.1145/3626772.3657891

GitHub: https://github.com/DataScienceUIBK/ChroniclingAmericaQA?tab=readme-ov-file

In [51]:
%pip install -r requirements.txt


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 26.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 653.1/653.1 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 770.6/770.6 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20/20 [spacy]m19/20 [spacy]]c]]ib]

[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: /opt/homebrew/opt/python@3.11/bin/python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [52]:
import os
os.makedirs("data", exist_ok=True)

!curl -L "https://huggingface.co/datasets/Bhawna/ChroniclingAmericaQA/resolve/main/test.json?download=true" -o data/test.json
!curl -L "https://huggingface.co/datasets/Bhawna/ChroniclingAmericaQA/resolve/main/train.json?download=true" -o data/train.json
!curl -L "https://huggingface.co/datasets/Bhawna/ChroniclingAmericaQA/resolve/main/dev.json?download=true" -o data/validation.json

import json

files = ["data/train.json", "data/validation.json", "data/test.json"]

for path in files:
    print(f"\n===== {path} =====")
    try:
        with open(path, "r", encoding="utf-8") as f:
            # Read a few hundred characters to see what kind of JSON it is
            head = f.read(500)
            print("Preview of first 500 characters:\n")
            print(head[:500])
        # Try to load only part of the file
        with open(path, "r", encoding="utf-8") as f:
            data = json.load(f)
        if isinstance(data, list):
            print(f"\nLoaded {len(data)} items (list).")
            print("Dictionary keys:", list(data[0].keys()))
            print(json.dumps(data[0], indent=2)[:600])
        elif isinstance(data, dict):
            print("\nTop-level is a dictionary. Keys:", list(data.keys()))
            for k, v in data.items():
                if isinstance(v, list):
                    print(f"Key '{k}' contains a list of {len(v)} items.")
                    if v:
                        print("First item keys:", list(v[0].keys()))
                        print(json.dumps(v[0], indent=2)[:600])
                        break
        else:
            print(f"Unexpected top-level type: {type(data)}")
    except Exception as e:
        print(f"Could not parse {path} as JSON: {e}")

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1354  100  1354    0     0      0      0 --:--:-- --:--:-- --:--:--     0    0   5390      0 --:--:-- --:--:-- --:--:--  5373
100 71.5M  100 71.5M    0     0  7810k      0  0:00:09  0:00:09 --:--:-- 11.4M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1360  100  1360    0     0   6211      0 --:--:-- --:--:-- --:--:--  6238
100 1315M  100 1315M    0     0  14.2M      0  0:01:32  0:01:32 --:--:-- 9296k4  0:00:58 14.6M   0  15.1M      0  0:01:27  0:01:21  0:00:06  9.8M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1344  100  1344    0     0   3979      0 --:--:-- --:--:-- --:--:--  3988
100 71.8M  1

# Create the Document Collection

To do that, we create a new json file that contains the 'para_id', 'context', 'raw_ocr', 'publication_date' keys, for all para_id in the collection.

para_id: is the id of a paragraph of a news paper page.

In [53]:
import json
import os

inputs = ["data/train.json", "data/validation.json", "data/test.json"]
output = "data/document_collection.json"

def load_list_or_empty(path):
    if not os.path.exists(path) or os.path.getsize(path) == 0:
        print(f"Skipping {path} because it is missing or empty")
        return []
    try:
        with open(path, "r", encoding="utf-8") as f:
            data = json.load(f)
        if isinstance(data, list):
            return data
        print(f"Skipping {path} because it is not a list at the top level")
        return []
    except json.JSONDecodeError:
        print(f"Skipping {path} because it is not valid JSON")
        return []

def project(recs):
    out = []
    for r in recs:
        out.append({
            "para_id": r.get("para_id", ""),
            "context": r.get("context", ""),
            "raw_ocr": r.get("raw_ocr", ""),
            "publication_date": r.get("publication_date", "")
        })
    return out

all_recs = []
for p in inputs:
    recs = load_list_or_empty(p)
    print(f"Loaded {len(recs)} records from {p}")
    all_recs.extend(project(recs))

# deduplicate by para_id keeping the first one seen
uniq = {}
for rec in all_recs:
    pid = rec.get("para_id", "")
    if pid and pid not in uniq:
        uniq[pid] = rec

result = list(uniq.values())

with open(output, "w", encoding="utf-8") as f:
    json.dump(result, f, ensure_ascii=False, indent=2)

print(f"Wrote {len(result)} records to {output}")
print(json.dumps(result[:3], indent=2))

Loaded 439302 records from data/train.json
Loaded 24111 records from data/validation.json
Loaded 24084 records from data/test.json
Wrote 131921 records to data/document_collection.json
[
  {
    "para_id": "New_Hampshire_18070804_1",
    "context": "Aiscellaneous Repository. From the Albany Register, WAR, OR A PROSPECT OF IT, From recent instances of British Outrage. BY: WILLIAM RAY, Author of the contemplated publication, entitled, \u201cHorrors of Slavery, or the American Turf in Tripoli,\u201d VOTARIES of Freedom, arm! The British Lion roars! Legions of Valor, take th\u2019 alarm\u2014; Rash, rush to guard our shores! Behold the horrid deed\u2014 Your brethren gasping lie! Beneath a tyrant\u2019s hand they bleed\u2014 They groan\u2014they faint\u2014they die. Veterans of seventy-six, Awake the slumbering sword;\u2014 Hearts of your murderous foes transfix\u2014 'Tis vengeance gives the word. Remember Lexington, And Bunker\u2019s tragic hill; \u201cThe same who spilt your blood there

## You should check that the collection you have matches that of the paper!

In [54]:
import pandas as pd
for path in inputs:
    with open(path, "r", encoding="utf-8") as f:
        data = json.load(f)
        df_check = pd.read_json(path)
        print(f'Shape of {path}: {df_check.shape}')

Shape of data/train.json: (439302, 11)
Shape of data/validation.json: (24111, 11)
Shape of data/test.json: (24084, 11)


The dimensions match the ones of the paper at https://github.com/DataScienceUIBK/ChroniclingAmericaQA

# Create the Test Queries Data Structure

We keep the first 10.000 queries due to memory errors in the free colab version.

To be comparable, please keep the top 10.000 queries for evaluation.

In [55]:
import json
import re
import unicodedata
import string

input_file = "data/test.json"
output_file = "data/test_queries.json"

# Load the data
with open(input_file, "r", encoding="utf-8") as f:
    data = json.load(f)

def clean_question(text):
    if not isinstance(text, str):
        return ""
    text = unicodedata.normalize("NFKC", text)
    text = re.sub(rf"[{re.escape(string.punctuation)}]", " ", text)  # remove punctuation
    text = re.sub(r"\s+", " ", text)  # collapse multiple spaces
    return text.strip()

# Extract and clean
queries = [
    {
        "query_id": item.get("query_id", ""),
        "question": clean_question(item.get("question", "")),
    }
    for item in data
]

# Sort by query_id (assuming numeric)
queries = sorted(queries, key=lambda x: int(x["query_id"]) if str(x["query_id"]).isdigit() else x["query_id"])

# Keep only the first 10,000
queries = queries[:10000]

# Save new JSON
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(queries, f, ensure_ascii=False, indent=2)

print(f"Saved {len(queries)} entries to {output_file}")
print(json.dumps(queries[:3], indent=2))

Saved 10000 entries to data/test_queries.json
[
  {
    "query_id": "test_1",
    "question": "How many lots did Thomas Peirce have"
  },
  {
    "query_id": "test_10",
    "question": "Who gave Hamilton the substance of what he had proposed on the part of General Hamilton"
  },
  {
    "query_id": "test_100",
    "question": "Who informs his FRIENDS and the PUBLIC that he has taken that justly celebrated INN in this city"
  }
]


# Create the Qrels for the test set

In [56]:
input_file = "data/test.json"
qrels_file = "data/test_qrels.json"
answers_file = "data/test_query_answers.json"

# Load the data
with open(input_file, "r", encoding="utf-8") as f:
    data = json.load(f)

# Build the qrels file: query_id, iteration=0, para_id, relevance=1
qrels = [
    {
        "query_id": item.get("query_id", ""),
        "iteration": 0,
        "para_id": item.get("para_id", ""),
        "relevance": 1
    }
    for item in data
]

# Build the query_answers file: same plus answer and org_answer
query_answers = [
    {
        "query_id": item.get("query_id", ""),
        "iteration": 0,
        "para_id": item.get("para_id", ""),
        "relevance": 1,
        "answer": item.get("answer", ""),
        "org_answer": item.get("org_answer", "")
    }
    for item in data
]

# Save both files
with open(qrels_file, "w", encoding="utf-8") as f:
    json.dump(qrels, f, ensure_ascii=False, indent=2)

with open(answers_file, "w", encoding="utf-8") as f:
    json.dump(query_answers, f, ensure_ascii=False, indent=2)

print(f"Saved {len(qrels)} entries to {qrels_file}")
print(f"Saved {len(query_answers)} entries to {answers_file}")
print("Sample qrels entry:", qrels[0])
print("Sample query_answers entry:", query_answers[0])

Saved 24084 entries to data/test_qrels.json
Saved 24084 entries to data/test_query_answers.json
Sample qrels entry: {'query_id': 'test_1', 'iteration': 0, 'para_id': 'New_Hampshire_18030125_16', 'relevance': 1}
Sample query_answers entry: {'query_id': 'test_1', 'iteration': 0, 'para_id': 'New_Hampshire_18030125_16', 'relevance': 1, 'answer': '183', 'org_answer': '183'}


# Retrieval

### Extract data from json files

In [57]:
input_files = ['data/document_collection.json', 'data/test.json', 'data/test_qrels.json', 'data/test_queries.json', 'data/test_query_answers.json', 'data/train.json', 'data/validation.json']

dataframes = {}
for input_file in input_files:
    with open(input_file, "r", encoding="utf-8") as f:
        data = json.load(f)
        dataframes[input_file] = pd.read_json(input_file)

In [58]:
dataframes['data/document_collection.json']

,para_id,context,raw_ocr,publication_date
0,New_Hampshire_18070804_1,Aiscellaneous Repository. From the Albany Regi...,fAiscellancous Bepogitory.\n. dvom the Albany ...,1807-08-04
1,New_Hampshire_18070804_4,Surely he above the rest of his fellow mortals...,Surely he a\nbove the rest of his fellow morta...,1807-08-04
2,New_Hampshire_18070804_5,"At Westmoreland, Mrs. Sally Lincoln, wife of M...","At Weltmoreland, Mrs. Sally Liacoln, wife\n~of...",1807-08-04
3,New_Hampshire_18070804_8,Upon the correction of this remedy the stomach...,tion of this remedy the flomach is invariably\...,1807-08-04
4,New_Hampshire_18070804_9,"Also FOR SALE AS ABOVE, NEW GOODS, STEPHEN HAR...","*°\n, ALSO POR SALE AS ABOVE,\no NEW-GEODS, -\...",1807-08-04
...,...,...,...,...
131916,Nebraska_19130626_7,"""Did you?” said Fran politely. “So father grad...","""Did you?” said Fran politely. “So\nfather gra...",1913-06-26
131917,Indiana_19170719_6,"When a boy begins to learn a trade, the ""play ...","When a boy begins to learn a trade, the ""play\...",1917-07-19
131918,Kentucky_19110727_5,It is situated in the valley of the great many...,It is ftltuaiod In tho val\nley of Uio great n...,1911-07-27
131919,Rhode_Island_19140626_10,"A PRACTICAL LESSON IN AGRICULTURE, MAY 1708, T...","A PRACITICA, ZESSQN 2V AGRICTZ,\nLEFY 170 RIGH...",1914-06-26


In [59]:
dataframes['data/train.json']

,query_id,question,answer,org_answer,para_id,context,raw_ocr,publication_date,trans_que,trans_ans,url
0,train_1,"Who is the author of the book, ""Horrors of Sla...",WILLIAM RAY,WILLIAM RAY,New_Hampshire_18070804_1,Aiscellaneous Repository. From the Albany Regi...,fAiscellancous Bepogitory.\n. dvom the Albany ...,1807-08-04,0,0,https://chroniclingamerica.loc.gov/lccn/sn8302...
1,train_2,Who was the Grand Officer of the Legion of Honor?,de Rosemberg,de Rosemberg,New_Hampshire_18070804_4,Surely he above the rest of his fellow mortals...,Surely he a\nbove the rest of his fellow morta...,1807-08-04,0,0,https://chroniclingamerica.loc.gov/lccn/sn8302...
2,train_3,What country was Gen. de Rosemberg formerly Ma...,France,France,New_Hampshire_18070804_4,Surely he above the rest of his fellow mortals...,Surely he a\nbove the rest of his fellow morta...,1807-08-04,0,0,https://chroniclingamerica.loc.gov/lccn/sn8302...
3,train_4,What was the title of Rev. Joseph McKean?,de Rosemberg,de Rosemberg,New_Hampshire_18070804_4,Surely he above the rest of his fellow mortals...,Surely he a\nbove the rest of his fellow morta...,1807-08-04,0,0,https://chroniclingamerica.loc.gov/lccn/sn8302...
4,train_5,Who was the wife of Mr. Spencer L. at Westmore...,Sally Lincoln,Sally Lincoln,New_Hampshire_18070804_5,"At Westmoreland, Mrs. Sally Lincoln, wife of M...","At Weltmoreland, Mrs. Sally Liacoln, wife\n~of...",1807-08-04,0,0,https://chroniclingamerica.loc.gov/lccn/sn8302...
...,...,...,...,...,...,...,...,...,...,...,...
439297,train_439298,Who is the pastor of First Baptist Church?,W. R. Bradshaw,W. R. Bradshaw,North_Carolina_19181130_6,"He went to school in France, received his comm...","He went\nto school in' France, received his\nc...",1918-11-30,0,0,https://chroniclingamerica.loc.gov/lccn/sn9106...
439298,train_439299,What day of the week is the school for Arthur ...,"November 24,1918",Sunday,North_Carolina_19181130_6,"He went to school in France, received his comm...","He went\nto school in' France, received his\nc...",1918-11-30,0,1,https://chroniclingamerica.loc.gov/lccn/sn9106...
439299,train_439300,How long has CHICHESTER'S DIAMOND BRAND PILLS ...,25 years,25 years,North_Carolina_19181130_7,There are plenty of such gifts; you'll find lo...,There are plenty of such gifts; yon'll find\nl...,1918-11-30,0,0,https://chroniclingamerica.loc.gov/lccn/sn9106...
439300,train_439301,On what day of the week is Trinity Lutheran sc...,"November 24,1918",Sunday,North_Carolina_19181130_7,There are plenty of such gifts; you'll find lo...,There are plenty of such gifts; yon'll find\nl...,1918-11-30,0,1,https://chroniclingamerica.loc.gov/lccn/sn9106...


**NOTE: in `data/document_collection.json` the rows are already deduplicated**

### _Preprocessing_

#### **Linguistic Processing**

##### Normalization
We lowercase everything and remove all special characters/tags

In [60]:
import pandas as pd
import re
import unicodedata

def normalize_text(val):
    if not isinstance(val, str):
        return val
    val = unicodedata.normalize('NFKC', val)
    val = val.lower()
    val = re.sub(r'<[^>]+>', ' ', val)
    val = re.sub(r'[^a-z0-9\s]', '', val)
    val = re.sub(r'\s+', ' ', val).strip()
    return val

target_key = 'data/document_collection.json'

if target_key in dataframes:
    print(f"Normalizzazione in corso per: {target_key}...")
    
    # ESTRAI il dataframe
    df = dataframes[target_key]
    
    # APPLICA la normalizzazione a tutte le celle
    # Nota: .applymap() esegue la funzione su ogni singola cella del DataFrame
    df_normalized = df.applymap(normalize_text)
    
    # AGGIORNA il dizionario con la versione pulita
    dataframes[target_key] = df_normalized
    
    print(dataframes[target_key].head()) # Controllo rapido
else:
    print(f"Error: The key '{target_key}' does not exist in the dictionary dataframes.")

Normalizzazione in corso per: data/document_collection.json...


/var/folders/1g/qh0zz9cn4_zbb4j4n0g57_dm0000gn/T/ipykernel_71642/893073684.py:25: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_normalized = df.applymap(normalize_text)


                 para_id                                            context  \
0  newhampshire180708041  aiscellaneous repository from the albany regis...   
1  newhampshire180708044  surely he above the rest of his fellow mortals...   
2  newhampshire180708045  at westmoreland mrs sally lincoln wife of mr s...   
3  newhampshire180708048  upon the correction of this remedy the stomach...   
4  newhampshire180708049  also for sale as above new goods stephen hardy...   

                                             raw_ocr publication_date  
0  faiscellancous bepogitory dvom the albany regi...         18070804  
1  surely he a bove the rest of his fellow mortal...         18070804  
2  at weltmoreland mrs sally liacoln wife of mrsp...         18070804  
3  tion of this remedy the flomach is invariably ...         18070804  
4  also por sale as above o newgeods stephen hard...         18070804  


##### NER
We want to identify named-entities before lemmatizing the text, so that we do not lose any entity by "shrinking" words to their base forms.

In [61]:
from transformers import AutoTokenizer, pipeline

MODEL_NAME = "impresso-project/ner-stacked-bert-multilingual-light"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

ner_pipeline = pipeline(model=MODEL_NAME, tokenizer=tokenizer, trust_remote_code=True, device="cpu")

doc_collection = dataframes['data/document_collection.json']

test_subset = doc_collection.head(25).copy()

results = []
for index, row in test_subset.iterrows():
    sentence = str(row['context'])
    entities = ner_pipeline(sentence, tokens=sentence.split())
    results.append(entities)
test_subset['ner_entities'] = results

/opt/homebrew/lib/python3.11/site-packages/torch/nn/modules/transformer.py:392: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(
Device set to use cpu


In [62]:
test_subset[['context', 'ner_entities']]

,context,ner_entities
0,aiscellaneous repository from the albany regis...,"[{'type': 'pers', 'confidence_ner': 0.41, 'ind..."
1,surely he above the rest of his fellow mortals...,"[{'type': 'pers', 'confidence_ner': 0.46, 'ind..."
2,at westmoreland mrs sally lincoln wife of mr s...,"[{'type': 'loc', 'confidence_ner': 0.87, 'inde..."
3,upon the correction of this remedy the stomach...,"[{'type': 'loc', 'confidence_ner': 0.67, 'inde..."
4,also for sale as above new goods stephen hardy...,"[{'type': 'loc', 'confidence_ner': 0.68, 'inde..."
5,at a meeting of the committee of the subscript...,"[{'type': 'org', 'confidence_ner': 0.56, 'inde..."
6,notice is hereby given to the proprietors of t...,"[{'type': 'loc', 'confidence_ner': 0.66, 'inde..."
7,lo l george frost esq x 30 i 14 2 1 3 3 theodo...,"[{'type': 'loc', 'confidence_ner': 0.58, 'inde..."
8,swedes do 150 bbls fresh flour molasses and ru...,"[{'type': 'loc', 'confidence_ner': 0.76, 'inde..."
9,the portsmouthaathinna in the miscellaneous re...,"[{'type': 'org', 'confidence_ner': 0.5, 'index..."


##### Lemmatization
Placed here to standardize semantically the sentences in the documents

In [63]:
import spacy
import pandas as pd

try:
    if 'nlp' not in locals():
        nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])
except OSError:
    from spacy.cli import download
    download("en_core_web_sm")
    nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])

target_key = 'data/document_collection.json'
target_column = 'context' 

if target_key in dataframes:
    print(f"🚀 Avvio lemmatizzazione ottimizzata (Batch Processing) su: {target_key}")
    df = dataframes[target_key]
    
    if target_column in df.columns:
        texts = df[target_column].astype(str).tolist()
        
        processed_texts = []
        
        print(f"Elaborazione di {len(texts)} documenti...")
        
        for doc in nlp.pipe(texts, batch_size=2000, n_process=1):
            lemmas = [token.lemma_ for token in doc if not token.is_space]
            processed_texts.append(" ".join(lemmas))
            
        new_col_name = f"{target_column}_lemma"
        df[new_col_name] = processed_texts
        
        dataframes[target_key] = df
        
        print(f"✅ Finito! Creata colonna: {new_col_name}")
        print(df[[target_column, new_col_name]].head())
        
    else:
        print(f"Errore: Colonna '{target_column}' non trovata. Controlla il nome esatto.")
else:
    print("DataFrame non trovato.")

Scaricamento modello spaCy...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 16.2 MB/s eta 0:00:00a 0:00:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: /opt/homebrew/opt/python@3.11/bin/python3.11 -m pip install --upgrade pip


Analisi colonne per lemmatizzazione su: data/document_collection.json
Colonne candidate trovate: ['para_id', 'context', 'raw_ocr', 'publication_date']
 -> Salto colonna 'para_id' (probabile ID)
 -> Lemmatizzazione in corso per: 'context'...


KeyboardInterrupt: 

##### N-gram based tokenization
Important to place it after normalization, in this tokenization can be integrated a NER-aware part so that "the tokenization is also entity-guided"

In [ ]:
import pandas as pd

def ner_aware_ngram_tokenizer(row, text_col='lemmatized_context', ner_col='ner_entities', n=2):
    """
    1. Prende il testo lemmatizzato.
    2. Usa le entità NER per 'incollare' le parole composte (New York -> new_york).
    3. Genera N-grams dal testo modificato.
    """
    text = row.get(text_col, "")
    entities = row.get(ner_col, [])
    
    if not isinstance(text, str) or not text.strip():
        return []
    
    # Entity Glueing (Incollaggio Entità)
    # Creiamo una versione del testo dove le entità sono unite da underscore.
    
    # Se abbiamo entità, proviamo a unirle nel testo
    if isinstance(entities, list) and len(entities) > 0:
        # Ordiniamo per lunghezza decrescente per evitare sostituzioni parziali
        try:

            entity_texts = []
            for ent in entities:
                if 'word' in ent:
                    entity_texts.append(ent['word'])
                elif 'entity_group' in ent:
                    entity_texts.append(ent['entity_group'])
                elif 'entity' in ent:
                    entity_texts.append(ent['entity']) 
            
            for ent_text in sorted(entity_texts, key=len, reverse=True):
                clean_ent = ent_text.lower().strip()
                if " " in clean_ent:
                    merged_ent = clean_ent.replace(" ", "_")
                    text = text.replace(clean_ent, merged_merged_ent)
        except Exception as e:
            pass

    # Tokenization Standard ---
    tokens = text.split() 
    
    # Generazione N-grams ---
    if len(tokens) < n:
        return []
        
    # Se n=2 (Bigrams): zip(tokens, tokens[1:])
    n_grams_tuples = zip(*[tokens[i:] for i in range(n)])
    
    # Unisce le tuple in stringhe: ("new_york", "is") -> "new_york is"
    n_grams_list = [" ".join(ngram) for ngram in n_grams_tuples]
    
    return n_grams_list

target_key = 'data/document_collection.json'
text_column = 'lemmatized_context' 
ner_column = 'ner_entities' 

if target_key in dataframes:
    print(f"Initiating N-gram Tokenization (Entity-Aware) on: {target_key}...")
    df = dataframes[target_key]
    
    if text_column in df.columns and ner_column in df.columns:
        
        N_VALUE = 2 
        
        print(f"Generating {N_VALUE}-grams...")
        
        df['ngrams'] = df.apply(
            lambda row: ner_aware_ngram_tokenizer(row, text_col=text_column, ner_col=ner_column, n=N_VALUE), 
            axis=1
        )

        dataframes[target_key] = df
        
        print(df[['lemmatized_context', 'ngrams']].head())
        
    else:
        print(f"Error: Columns '{text_column}' or '{ner_column}' missing. Check names.")
else:
    print(f"Error: {target_key} not found.")

### _Multi-field Indexing_

## Phase I

### **BM25 Retrieval from raw OCR (baseline 1)**

### **BM25 Retrieval from corrected OCR (baseline 2)**

### **BM25 Retrieval from both raw and corrected OCR using RRF formula (baseline 3)**